# RNN_long_char
다음번에 나올 char 맞추기

In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn

tf.set_random_seed(777)  # reproducibility

sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

char_set = list(set(sentence))
char_dic = {w: i for i, w in enumerate(char_set)}

# data_dim = len(char_set)
hidden_size = 20
num_classes = len(char_set)
sequence_length = 10  # Any arbitrary number
learning_rate = 0.1
print('num_classes', num_classes)

dataX = []
dataY = []
for i in range(len(sentence) - sequence_length):
    x_str = sentence[i:i+sequence_length]
    y_str = sentence[i+1:i+sequence_length+1]
    if i % 20 == 0:
        print(i, x_str, '->', y_str)

    x = [char_dic[c] for c in x_str]  # x str to index
    y = [char_dic[c] for c in y_str]  # y str to index

    dataX.append(x)
    dataY.append(y)

batch_size = len(dataX)

num_classes 25
0 if you wan -> f you want
20  a ship, d -> a ship, do
40 up people  -> p people t
60 o collect  ->  collect w
80 on't assig -> n't assign
100 ks and wor -> s and work
120 her teach  -> er teach t
140 ng for the -> g for the 
160 mmensity o -> mensity of


In [3]:
X = tf.placeholder(tf.int32, [None, sequence_length])
Y = tf.placeholder(tf.int32, [None, sequence_length])

# One-hot encoding
X_one_hot = tf.one_hot(X, num_classes)
print(X_one_hot)  # check out the shape

# Make a lstm cell with hidden_size (each unit output vector size)
def lstm_cell():
    cell = rnn.BasicLSTMCell(hidden_size, state_is_tuple=True)
    return cell

# LSTM을 쌓기 위해 사용.
# LSTM layer 하나만 사용할꺼면 그냥 BasicLSTMCell을 dynamic_rnn으로 바로 넘기면 됨.
multi_cells = rnn.MultiRNNCell([lstm_cell() for _ in range(2)], state_is_tuple=True)

# outputs: unfolding size x hidden size, state = hidden size
outputs, _states = tf.nn.dynamic_rnn(multi_cells, X_one_hot, dtype=tf.float32)

# FC layer
X_for_fc = tf.reshape(outputs, [-1, hidden_size])
outputs = tf.contrib.layers.fully_connected(X_for_fc, num_classes, activation_fn=None)

# reshape out for sequence_loss
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])
# 여기에 softmax도 넣어주는게 더 낫지 않을까?

# All weights are 1 (equal weights)
weights = tf.ones([batch_size, sequence_length])

# sequence output의 loss를 계산해주는 편한 방법.
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=Y, weights=weights)
mean_loss = tf.reduce_mean(sequence_loss)
train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(mean_loss)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(501):
    _, l, results = sess.run([train_op, mean_loss, outputs], feed_dict={X: dataX, Y: dataY})
    if i % 100 == 0:
        index = np.argmax(results[0], axis=1)
        print(i, ''.join([char_set[t] for t in index]), l)

# Let's print the last char of each result to check it works
results = sess.run(outputs, feed_dict={X: dataX})
for j, result in enumerate(results):
    index = np.argmax(result, axis=1)
    if j is 0:  # print all for the first result to make a sentence
        print(''.join([char_set[t] for t in index]), end='')
    else:
        print(char_set[index[-1]], end='')

sess.close()

Tensor("one_hot:0", shape=(?, 10, 25), dtype=float32)
0 skhggggbbb 3.2196846
100 l you want 0.24567482
200 f you want 0.23203954
300 l you want 0.23047268
400 t you want 0.229613
500 l you want 0.22924514
l you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.